
# Implementing Stragtegy 3 of the CNN-RNN approach: 

## Import Statements

In [ ]:
pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
# import IPython.display as display
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input
import numpy as np




## Mount the drive and load the data. Final Data will be Image,SuperClass Label, Subclass Label


1.   We created a train, validation, test split to avoid over fitting
2.   All our models follow the same split and are evealuated against the same test data.



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!ls /content/gdrive/MyDrive/data/data_split/test

In [ ]:
# DATA
BATCH_SIZE = 32
BUFFER_SIZE = BATCH_SIZE * 2
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (8, 8, 3)
NUM_CLASSES = 3 # super class

# AUGMENTATION
IMAGE_SIZE = 64
INPUT_SIZE = 8
IMG_SIZE = 32

In [ ]:
import pandas as pd
DATA_DIR = '/content/gdrive/MyDrive/data/'
train_label_csv = pd.read_csv(DATA_DIR + 'train_data.csv')
test_label_csv = pd.read_csv(DATA_DIR+'data_split/Truth.csv')
train_label_csv['subclass_index'].max()

88

In [ ]:
train_label_csv.head()

,image,superclass_index,subclass_index
0,0.jpg,1,63
1,1.jpg,0,24
2,2.jpg,0,74
3,3.jpg,2,72
4,4.jpg,2,43


In [ ]:
import glob
class0 = glob.glob(DATA_DIR + 'data_split/train/superclass/0/*.*')
data = []
labels = []
labels_subclass = []

for i in class0:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (32,32), interpolation='bicubic')
    image=np.array(image)
    data.append(image)
    labels.append(0)
    i=i.replace(DATA_DIR+"data_split/train/superclass/0/","")
    df = train_label_csv[train_label_csv['image']==i]['subclass_index'].values[0]
    labels_subclass.append(int(df))

In [ ]:
class1 = glob.glob(DATA_DIR+'data_split/train/superclass/1/*.*')
for i in class1:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (32,32), interpolation='bicubic')
    image=np.array(image)
    data.append(image)
    labels.append(1)
    i=i.replace(DATA_DIR+"data_split/train/superclass/1/","")
    i = i.replace(" (1)","")
    i = i.replace("(1)","")
    i = i.replace("(2)","")
    i = i.replace("(3)","")
    i = i.replace("(4)","")
    df = train_label_csv[train_label_csv['image']==i]['subclass_index'].values[0]
    labels_subclass.append(int(df))

In [ ]:
class3 = glob.glob(DATA_DIR+'data_split/train/superclass/2/*.*')
for i in class3:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (32,32), interpolation='bicubic')
    image=np.array(image)
    data.append(image)
    labels.append(2)
    i=i.replace(DATA_DIR+"data_split/train/superclass/2/","")
    i = i.replace(" (1)","")
    i = i.replace("(1)","")
    df = train_label_csv[train_label_csv['image']==i]['subclass_index'].values[0]
    labels_subclass.append(int(df))

In [ ]:
y = [[labels[i],labels_subclass[i]] for i in range(len(labels_subclass))]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data, y, test_size=0.3, random_state=42,stratify=labels_subclass,shuffle=True)

In [ ]:
import glob
test = glob.glob(DATA_DIR+'data_split/test/*.jpg')
t_data = []
t_labels = []
t_labels_subclass = []

for i in test:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (32,32), interpolation='bicubic')
    image=np.array(image)
    t_data.append(image)
    i=i.replace(DATA_DIR+"data_split/test/","")
    df = test_label_csv[test_label_csv['image']==i]['subclass_index'].values[0]
    df_super = test_label_csv[test_label_csv['image']==i]['superclass_index'].values[0]
    t_labels.append(int(df_super))
    t_labels_subclass.append(int(df))

In [ ]:
x_test = t_data
y_test = [[t_labels[i],t_labels_subclass[i]] for i in range(len(t_labels_subclass))]

In [ ]:
import numpy as np
x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

## Model and Evaluation


*   Since the given images are 8x8, we chose Vgg16 as the base model as it required minimum resizing
*   Based on few expreiments we noticed as we resize more, the accuracy decreases.

Approach

*   Our network has two parallel paths after a point one which leads classifies super class and one that classifies subclass
*   We compute loss by providing more weight to the the subclass loss. This is because we have 89 subcasses and only 3 super classes. We experimented by giving the same weights to both but that led to an extremely slow improvement in the subclass accuracy.





In [ ]:
input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
class CNNFeatureExtractor(keras.Model):
  def __init__(self,input_tensor,num_super_class,num_sub_class,IMG_SIZE):
    super().__init__()
    self.rescale = layers.Rescaling(1./255)
    self.base_model = VGG16(input_tensor=input_tensor,weights='imagenet', include_top=False)
    self.base_model.trainable = False

    self.global_averaging_layer = layers.GlobalAveragePooling2D()
    self.Dense1 = layers.Dense(256,activation='relu')
    self.dropout1 = tf.keras.layers.Dropout(0.5)
    self.Dense2 = layers.Dense(128,activation='relu')
    self.dropout2 = tf.keras.layers.Dropout(0.5)
    self.Dense3 = layers.Dense(96,activation='relu')
    self.Classifier1 = layers.Dense(num_super_class, activation='softmax',name='output_1')
    self.Classifier2 = layers.Dense(num_sub_class, activation='softmax',name='output_2')

    
    
  
  def call(self,input,training=False):
    input = self.rescale(input)
    input_1 = self.base_model(input)
    input_2 = self.global_averaging_layer(input_1)
    input_3 = self.Dense1(input_2)
    input_3 = self.dropout1(input_3,training)
    input_4 = self.Dense2(input_3)
    input_4 = self.dropout2(input_4,training)
    input_5 = self.Dense3(input_4)
    super_out = self.Classifier1(input_4)
    sub_out = self.Classifier2(input_5)
    
    
    return super_out,sub_out


In [ ]:
cnn = CNNFeatureExtractor(input_tensor,3,89,32)

We Initally keep trainable as False and train the network for 20 epochs by freezing the base model. Later on, we unfreeze all the layers and retrain for 10 more epochs with a lower learning rate. This gave an increased the accuracy by 4%. From 14% to 18%.

In [ ]:
cnn.trainable=True

In [ ]:
loss_function    =  {'output_1':tf.keras.losses.SparseCategoricalCrossentropy(), 'output_2':tf.keras.losses.SparseCategoricalCrossentropy()}
loss_weight =  {'output_1':0.2, 'output_2':0.8}

In [ ]:
#learning_rate=1e-5
optimizer=keras.optimizers.Adam(learning_rate=1e-4)
cnn.compile(optimizer=optimizer,
              loss=loss_function,
              loss_weights=loss_weight,
              metrics=['accuracy'])

In [ ]:
output_values = {
    'output_1':y_train[:,0].reshape(-1,1),
    'output_2':y_train[:,1].reshape(-1,1)  
}

In [ ]:
cnn.fit(x_train,output_values,validation_data = (x_val, [y_val[:,0].reshape(-1,1), y_val[:,1].reshape(-1,1)]),
        epochs=15)

Epoch 1/15
101/101 [==============================] - 3s 33ms/step - loss: 2.7108 - output_1_loss: 0.4111 - output_2_loss: 3.2858 - output_1_accuracy: 0.8506 - output_2_accuracy: 0.1482 - val_loss: 2.8706 - val_output_1_loss: 0.4938 - val_output_2_loss: 3.4647 - val_output_1_accuracy: 0.8030 - val_output_2_accuracy: 0.1356
Epoch 2/15
101/101 [==============================] - 3s 34ms/step - loss: 2.6147 - output_1_loss: 0.3736 - output_2_loss: 3.1749 - output_1_accuracy: 0.8753 - output_2_accuracy: 0.1621 - val_loss: 2.8450 - val_output_1_loss: 0.4934 - val_output_2_loss: 3.4329 - val_output_1_accuracy: 0.8189 - val_output_2_accuracy: 0.1414
Epoch 3/15
101/101 [==============================] - 3s 32ms/step - loss: 2.5619 - output_1_loss: 0.3542 - output_2_loss: 3.1138 - output_1_accuracy: 0.8781 - output_2_accuracy: 0.1699 - val_loss: 2.7554 - val_output_1_loss: 0.4769 - val_output_2_loss: 3.3251 - val_output_1_accuracy: 0.8081 - val_output_2_accuracy: 0.1631
Epoch 4/15
101/101 [=====

In [ ]:
cnn.evaluate(x_test,[y_test[:,0].reshape(-1,1), y_test[:,1].reshape(-1,1)])

61/61 [==============================] - 1s 15ms/step - loss: 3.4873 - output_1_loss: 0.7885 - output_2_loss: 4.1620 - output_1_accuracy: 0.8095 - output_2_accuracy: 0.1756


[3.487278938293457,
 0.7885410785675049,
 4.16196346282959,
 0.809474766254425,
 0.17559216916561127]

## Evalauation on Model Share Data

In [ ]:
superclass_idx_to_name_map = pd.read_csv(DATA_DIR + 'super_classes_mapping.csv')
superclass_idx_to_name_map = {k:v for k,v in superclass_idx_to_name_map.values}
subclass_idx_to_name_map = pd.read_csv(DATA_DIR + 'sub_classes_mapping.csv')
subclass_idx_to_name_map = {k:v for k,v in subclass_idx_to_name_map.values}

In [ ]:
# !pip install aimodelshare --upgrade

In [ ]:
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/neuralnet_competition_data-repository:latest')


Data downloaded successfully.


In [ ]:
! unzip neuralnet_competition_data/X_test.zip

Streaming output truncated to the last 5000 lines.
  inflating: test_shuffle/8701.jpg   
  inflating: test_shuffle/1057.jpg   
  inflating: test_shuffle/6738.jpg   
  inflating: test_shuffle/7426.jpg   
  inflating: test_shuffle/8715.jpg   
  inflating: test_shuffle/3640.jpg   
  inflating: test_shuffle/3898.jpg   
  inflating: test_shuffle/5231.jpg   
  inflating: test_shuffle/2238.jpg   
  inflating: test_shuffle/4891.jpg   
  inflating: test_shuffle/5557.jpg   
  inflating: test_shuffle/4649.jpg   
  inflating: test_shuffle/3126.jpg   
  inflating: test_shuffle/7340.jpg   
  inflating: test_shuffle/8073.jpg   
  inflating: test_shuffle/1731.jpg   
  inflating: test_shuffle/2210.jpg   
  inflating: test_shuffle/4661.jpg   
  inflating: test_shuffle/7368.jpg   
  inflating: test_shuffle/6076.jpg   
  inflating: test_shuffle/1719.jpg   
  inflating: test_shuffle/6710.jpg   
  inflating: test_shuffle/3668.jpg   
  inflating: test_shuffle/4107.jpg   
  inflating: test_shuffle/5219.jpg   

In [ ]:
from tqdm import tqdm
def get_test_dataloader(image_size):
    # Load train data
    file_paths = [('test_shuffle/' + str(i) + '.jpg') for i in range(0, 9127)]
    images = [np.array(keras.preprocessing.image.load_img(i, color_mode='rgb', 
                                                 grayscale=False,
                                                 target_size=image_size, 
                                                 )) 
        for i in tqdm(file_paths, desc='Loading images')]

    images = np.array(images)
    return images


In [ ]:
modelshare_test_ds = get_test_dataloader((32,32))

Loading images: 100%|██████████| 9127/9127 [00:01<00:00, 8456.87it/s]


In [ ]:
len(modelshare_test_ds)

9127

In [ ]:
cnn2 = keras.models.load_model(DATA_DIR+'/model')

In [ ]:
pred_chk_super, pred_chk_sub = cnn2.predict(modelshare_test_ds)

286/286 [==============================] - 2s 8ms/step


In [ ]:
super_pred_max = np.argmax(pred_chk_super,axis=1)

In [ ]:
super_pred = [superclass_idx_to_name_map[i] for i in super_pred_max]

In [ ]:
super_pred_df = pd.DataFrame({'predictions': super_pred})

In [ ]:
super_pred_df

,predictions
0,dog
1,reptile
2,reptile
3,reptile
4,bird
...,...
9122,bird
9123,dog
9124,dog
9125,dog


In [ ]:
super_pred_df.to_csv('superclass_modelshare_test.csv', index=False)

In [ ]:
subclass_dataframe = pd.DataFrame(zip(np.argmax(pred_chk_sub,axis=1), np.max(pred_chk_sub,axis=1)))

In [ ]:
subclass_dataframe.loc[subclass_dataframe[1]<0.6,0]=89

In [ ]:
pred_sub_new = subclass_dataframe[0]

In [ ]:
sub_pred = [subclass_idx_to_name_map[i] for i in pred_sub_new]

In [ ]:
sub_pred_df = pd.DataFrame({'predictions': sub_pred})

In [ ]:
sub_pred_df

,predictions
0,novel
1,novel
2,novel
3,novel
4,ptarmigan
...,...
9122,novel
9123,novel
9124,novel
9125,novel


In [ ]:
sub_pred_df.to_csv('subclass_test_modelshare.csv', index=False)

## Results:
Super Class Accuracy: 71.62%
Sub Class Accuracy with Threshold 60% for novel classes: 70.22%
It seems like most of the cases in subclass provided in model share are novel classes. Hence the higher threshold entails higher accuracy. 